Zero-shot and few-shot learning en un problema de clasificación con instruction-tuned LLM
===================================================================

Los grandes modelos de lenguaje exhiben grandes habilidades en zero-shot learning. Sin embargo, los resultados dependen mucho de la capacidad del modelo, y de la ténica que utilicemos para resolver el problema.

En este ejemplo, utizaremos un modelo de lenguaje para resolver el problema de clasificación de tweets sin entrenar ningún modelo (zero-shot).

## Introdución

Los grandes modelos de lenguaje son capaces de resolver problemas de clasificación al utilizar determinadas estructuras del idioma. Algunos modelos de lenguaje están especificamente entrenados para seguir instrucciones, los cuales los hace muy útiles a la hora de implementar zero-shot or few-shot learning.

En este ejemplo, veremos como utilizar un modelo de aprendizaje automático entrenado de esta forma para resolver problemas de clasificación.

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/

!pip -q install transformers[torch] accelerate datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.6 MB/s eta 0:00:00


### Verificando el hardware disponible

In [ ]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print("Este notebook se está ejecutando en", device)

Este notebook se está ejecutando en cuda


## Trabajando con LLMs entrenados para instrucciones

En este ejemplo, utilizaremos el modelo `dolly` en su version de 2.8 millones de parámetros. Para poder utilizar este modelo en Google Colab, necesitamos realizar algunas optimizaciones, entre las cuales, bajar la precisión numérica a 16 bits.

In [ ]:
from transformers import pipeline

dolly = pipeline(
    model="databricks/dolly-v2-2-8b",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
  )

A new version of the following files was downloaded from https://huggingface.co/databricks/dolly-v2-2-8b:
- instruct_pipeline.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Podemos verificar como funciona este modelo:

In [ ]:
dolly("Explain to me the difference between nuclear fission and fusion.")

[{'generated_text': 'Nuclear fission is the process in a nuclear reactor whereby two or more protons or other particles collide with sufficient energy to split one proton into two lower energy fragments. This type of fission reaction is only possible within atomic nuclei due to the finite size of the atomic nuclei. Nuclear fusion, on the other hand, is the process by which nucleons (protons or neutrons) merge together to form heavier nuclei and give off additional energy in the form of photon(s). Nuclear fusion is possible in two circumstances: in stars (also called nuclear fusion bombs), where stars burn hydrogen in their core to produce energy via nuclear fusion, and in the core of atoms where the separation of the electrons is reduced to a scale much smaller than the nuclear diameter and allowed to interact much more easily.'}]

### ¿Que diferencia hay con modelos de lenguaje tradicionales?

Los modelos que están entrenados para seguir instrucciones, en general, fueron sometidos a un proceso de fine-tuning con conjunto de datos que disponen de instrucciones en determinado formato.

En el caso de Dolly, cada vez que lo ejecutamos, en realidad enviamos un texto distinto al modelo. El siguiente código nos permite ver exactamente que es lo que el modelo utiliza como data de entrada:

In [ ]:
import inspect
from pathlib import Path
from importlib.machinery import SourceFileLoader

source_file = inspect.getfile(dolly.__class__)
module_name = Path(source_file).stem
loader = SourceFileLoader(module_name, source_file)
modulevar = loader.load_module()

In [ ]:
print(getattr(modulevar, "PROMPT_FOR_GENERATION_FORMAT"))

Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Response:



En este texto, `{instructions}` es la posición donde nuestro texto termina siendo incrustado para generar lo que se conoce como el **prompt**. Note como `### Response` es utilizado para indicar al modelo que lo que continua es la respuesta a nuestra instrucción.

El modelo completará esta secuencia de texto hasta que se cumpla alguna de las condiciones de salida para la generación. En general, los modelos de lenguaje autoregresivos, generan tokens hasta alcanzar la cantidad máxima de tokens o hasta encontrar un token de finalización. En este caso, el token de finalización es:

In [ ]:
getattr(modulevar, "END_KEY")

'### End'

### Zero-shot learning

En general, para resolver cualquier tarea necesitaremos generar un texto o prompt que haga que el modelo genere texto que resuelve el problema de negocio:

In [ ]:
prompt = "Decide whether the following product review's sentiment is positive, neutral, or negative.\n\nProduct review:\n{}\nSentiment:"
print(prompt)


Decide whether the following product review's sentiment is positive, neutral, or negative.

Product review:
{}
Sentiment:


Ahora, supongamos que nuestro texto es como sigue:

In [ ]:
sample = "The reactivity of your team has been good...ish. However, the overal experience is questionable."

Podemos generar la respuesta del modelo como sigue:

In [ ]:
dolly(prompt.format(sample))

[{'generated_text': 'Negative'}]

### Few-shot learning

En few-shot learning, intentaremos generar un prompt en el cual multiples ejemplos se mencionan antes de solicitarle al modelo que complete el prompt:

In [ ]:
base_prompt = "Decide whether the following product reviews' sentiment is positive, neutral, or negative."
examples = [
    (
        "I love my new chess board!",
        "positive"
    ),
    (
        "Not what I expected but I guess it'll do",
        "neutral"
    ),
    (
        "I'm so disappointed. The product seemed much better on the website",
        "negative"
    )
]

Podemos generar nuestro prompt entonces como sigue:

In [ ]:
def generate_prompt(base_prompt, examples, sample_name, prediction_name, sample=None):
    prompt = base_prompt
    for example in examples:
        prompt += f"\n\n{sample_name}:\n'{example[0]}'\n{prediction_name}:\n{example[1]}"

    prompt += f"\n\n{sample_name}:\n'{{}}'\n{prediction_name}:\n"

    if sample:
        return prompt.format(sample)
    return prompt

In [ ]:
prompt = generate_prompt(base_prompt, examples, "Product review", "Sentiment", sample)
print(prompt)

Decide whether the following product reviews' sentiment is positive, neutral, or negative.

Product review:
I love my new chess board!
Sentiment:
positive

Product review:
Not what I expected but I guess it'll do
Sentiment:
neutral

Product review:
I'm so disappointed. The product seemed much better on the website
Sentiment:
negative

Product review:
The reactivity of your team has been good...ish. However, the overal experience is questionable.
Sentiment:



Podemos ver como funciona el modelo:

In [ ]:
dolly(prompt)

[{'generated_text': 'Product review:\nThe product review is neutral.\n\nProduct review:\nThe sentiment is negative.'}]

Podemos ver que en este caso el modelo genero un texto que es bastante mas largo del que esperabamos. Nos interesaria que solo mencione el sentimiento. Podemos mejorar nuestras instrucciones para solicitar al modelo que realize esto, o simplemente buscar especificamente por tokens en particular:

#### Utilizando etiquetas

Primero, utilizaremos el `tokenizer` para generar los IDs correspondiente a los tokens de las etiquetas que disponemos:

In [ ]:
target_labels = ['positive', 'neutral', 'negative']
target_token_ids = [dolly.tokenizer.encode(k)[0] for k in target_labels]

Generaremos una función que solo obtiene la probabilidad más alta para los tokens que están representados por las etiquetas que tenemos. Esta técnica supone que, a pesar de que el modelo genera un texto diferente, la probabilidad de la clase correcta siempre es más alta a lo largo de todo el texto generado.

In [ ]:
def generate_for_classification(pipeline, prompt, target_labels, target_token_ids):
    input_ids = pipeline.tokenizer(prompt, return_tensors="pt", padding=True).input_ids
    with torch.no_grad():
        outputs = pipeline.model(input_ids.to(pipeline.model.device))
        result = torch.nn.Softmax(dim=-1)(outputs.logits[:, -1, target_token_ids])

        predicted_token_ids = torch.argmax(result, axis=1)
        return [target_labels[i] for i in predicted_token_ids], outputs

Ejecutamos la función:

In [ ]:
predictions, outputs = generate_for_classification(dolly, prompt, target_labels, target_token_ids)

In [ ]:
predictions

['neutral']

En la función, estamos devolviendo también la variable `outputs` que contiene los tensores tal como son generados por el modelo. Esto lo estamos haciendo solo por fines educativos. Veamos como luce la salida:

In [ ]:
outputs.logits.shape

torch.Size([1, 114, 50280])

`50280` es la cantidad de palabras del diccionario:

In [ ]:
dolly.tokenizer.vocab_size

50254

## Resolviendo un problema de clasificación con few-shot learning

Cargamos el conjunto de datos:

In [ ]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

Trataremos de resolver entonces el mismo problema de clasificación con el que veniamos trabajando: clasificar los tweets dependiendo del sector al que pertenecen. Recordemos que tenemos 7 categorias distintas.

Mejoraremos un poco los nombres de las categorías:

In [ ]:
tweets['SECTOR'] = tweets['SECTOR'].map({
    "ALIMENTACION": "alimentos",
    "AUTOMOCION": "automobiles",
    "BANCA": "bancos",
    "BEBIDAS": "bebidas",
    "DEPORTES": "deportes",
    "RETAIL": "supermercados",
    "TELCO": "telefonía"
})

In [ ]:
labels = tweets['SECTOR'].unique().tolist()
labels

['supermercados',
 'telefonía',
 'alimentos',
 'automobiles',
 'bancos',
 'bebidas',
 'deportes']

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(tweets, test_size=0.33, stratify=tweets['SECTOR'])

In [ ]:
base_prompt = f"Clasificar los siguientes tweets según el producto o categoría a la que hacen referencia. Las posibles categorías son: {', '.join(labels)}"

In [ ]:
examples_df = train.groupby('SECTOR').apply(lambda x: x.sample(1)).reset_index(drop=True)
examples = [(row[1]['TEXTO'], row[1]['SECTOR']) for row in examples_df.iterrows()]

In [ ]:
prompt = generate_prompt(base_prompt, examples, "Tweet", "Categoría", sample=None)
print(prompt)

Clasificar los siguientes tweets según el producto o categoría a la que hacen referencia. Las posibles categorías son: supermercados, telefonía, alimentos, automobiles, bancos, bebidas, deportes

Tweet:
'¡Qué Oso, Bimbo! Filosofema 34 del año para irnos poniendo al corriente. https://t.co/xtJtTWlswb'
Categoría:
alimentos

Tweet:
'#colombia Toyota y Suzuki acuerdan una alianza para intercambiar tecnología y componentes https://t.co/ot4jjjoRNB'
Categoría:
automobiles

Tweet:
'#marketing “Porque el tiempo pasa volando, necesitas un plan de pensiones”, la nueva campaña de Bankia https://t.co/Xsj79tK9PB'
Categoría:
bancos

Tweet:
'Porque no hay nada como un buen serranito con patatas y una @Cruzcampo, juro. #Sevilla https://t.co/nWDSjbpyqt'
Categoría:
bebidas

Tweet:
'Gracias porque #YoSoyLaDivinidadEnAcción #YoSoyTú/ Corrió 7.00 km con Nike⁠+ Run Club🙏💕🖐🏿 https://t.co/QoguoxoRfg'
Categoría:
deportes

Tweet:
'Amor eterno al té de melocotón del Mercadona.'
Categoría:
supermercados

Tweet:
'#

Veamos como funciona con un ejemplo:

In [ ]:
sample = test.sample(1).iloc[0]

In [ ]:
sample['TEXTO']

'@sergiooliveiram ¿Peugeot 208, Kia Río o SEAT Ibiza? \nVersiones de aprox. 250-260k\nExcelentes vídeos.'

### Utilizando el modelo generativo

In [ ]:
dolly(prompt.format(sample['TEXTO']))

[{'generated_text': 'supermercados\nautomobiles\nbancos\nbebidas\ndeportes'}]

### Utilizando la probabilidad de las etiquetas

Primero, utilizaremos el `tokenizer` para generar los IDs correspondiente a los tokens de las etiquetas que disponemos:

In [ ]:
target_token_ids = [dolly.tokenizer.encode(k)[0] for k in labels]

Utilizamos ahora nuestra función `generate_for_classification`:

In [ ]:
predictions, outputs = generate_for_classification(dolly, prompt.format(sample['TEXTO']), labels, target_token_ids)

Verificamos la predicción:

In [ ]:
sample['SECTOR']

'automobiles'